In [1]:
import sys

if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = [sys.argv[0]]

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from src.models import *
from src.constants import *
from src.plotting import *
from src.pot import pot_eval
from src.utils import *
from src.diagnosis import *
from src.merlin import *
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
from time import time
from pprint import pprint

import warnings

warnings.filterwarnings('ignore')

Using backend: pytorch


### Importing data

In [3]:
columns = [
    'data:zone_air_heating_temperature_setpoint', 
    'data:zone_air_temperature_sensor', 
    'data:zone_air_cooling_temperature_setpoint', 
    'data:supply_air_flowrate_sensor', 
    'data:supply_air_damper_percentage_command', 
    'data:supply_air_flowrate_setpoint', 
    'dow',
    'hod',
    'class_label']

df = pd.read_csv('data/anomaly_detection_sample_1577622599.csv', usecols=columns)

In [4]:
df.head()

,data:zone_air_heating_temperature_setpoint,data:zone_air_temperature_sensor,data:zone_air_cooling_temperature_setpoint,data:supply_air_flowrate_sensor,data:supply_air_damper_percentage_command,data:supply_air_flowrate_setpoint,dow,hod,class_label
0,294.259259,296.037037,297.037037,0.044528,47.265250,0.037756,0,17,1
1,294.259259,294.925926,297.037037,0.203740,4.489845,0.188779,0,17,1
2,294.259259,296.648148,297.037037,0.036790,20.409615,0.035396,0,17,1
3,294.259259,296.425926,297.037037,0.154288,37.755330,0.151023,0,17,1
4,294.259259,296.092593,297.037037,0.178737,16.167740,0.169901,0,17,1


In [5]:
new_column_names = [
    'heating_setpoint', 
    'air_temperature', 
    'cooling_setpoint', 
    'supply_flowrate', 
    'damper_percentage', 
    'flowrate_setpoint', 
    'day_of_week', 
    'hour_of_day', 
    'label'
]

df.columns = new_column_names

In [6]:
df.head()

,heating_setpoint,air_temperature,cooling_setpoint,supply_flowrate,damper_percentage,flowrate_setpoint,day_of_week,hour_of_day,label
0,294.259259,296.037037,297.037037,0.044528,47.265250,0.037756,0,17,1
1,294.259259,294.925926,297.037037,0.203740,4.489845,0.188779,0,17,1
2,294.259259,296.648148,297.037037,0.036790,20.409615,0.035396,0,17,1
3,294.259259,296.425926,297.037037,0.154288,37.755330,0.151023,0,17,1
4,294.259259,296.092593,297.037037,0.178737,16.167740,0.169901,0,17,1


In [7]:
df['label'] = df['label'].replace({0: 1, 1: 0})

In [8]:
df.head()

,heating_setpoint,air_temperature,cooling_setpoint,supply_flowrate,damper_percentage,flowrate_setpoint,day_of_week,hour_of_day,label
0,294.259259,296.037037,297.037037,0.044528,47.265250,0.037756,0,17,0
1,294.259259,294.925926,297.037037,0.203740,4.489845,0.188779,0,17,0
2,294.259259,296.648148,297.037037,0.036790,20.409615,0.035396,0,17,0
3,294.259259,296.425926,297.037037,0.154288,37.755330,0.151023,0,17,0
4,294.259259,296.092593,297.037037,0.178737,16.167740,0.169901,0,17,0


In [9]:
df.shape

(60425, 9)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60425 entries, 0 to 60424
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   heating_setpoint   60425 non-null  float64
 1   air_temperature    60425 non-null  float64
 2   cooling_setpoint   60425 non-null  float64
 3   supply_flowrate    60425 non-null  float64
 4   damper_percentage  60425 non-null  float64
 5   flowrate_setpoint  60425 non-null  float64
 6   day_of_week        60425 non-null  int64  
 7   hour_of_day        60425 non-null  int64  
 8   label              60425 non-null  int64  
dtypes: float64(6), int64(3)
memory usage: 4.1 MB


### Data preparation

In [11]:
X = df.drop(['day_of_week', 'hour_of_day', 'label'], axis=1)
# X = df.drop(['label'], axis=1)
y = df['label']

In [12]:
X.shape

(60425, 6)

In [13]:
X_train, X_test, y_test = X.iloc[:27113, :], X.iloc[27113:, :], y.iloc[27113:]

In [14]:
X_train.tail(2)

,heating_setpoint,air_temperature,cooling_setpoint,supply_flowrate,damper_percentage,flowrate_setpoint
27111,288.703704,295.592593,302.592593,0.026422,99.9756,0.049554
27112,285.925926,297.537037,302.592593,0.079202,0.0000,0.023597


In [15]:
X_test.head(2)

,heating_setpoint,air_temperature,cooling_setpoint,supply_flowrate,damper_percentage,flowrate_setpoint
27113,288.703704,295.592593,302.592593,0.025942,99.9756,0.049554
27114,288.703704,296.592593,302.592593,-0.006073,100.0000,0.047195


In [16]:
X_train = X_train.values
X_test = X_test.values

In [17]:
def normalize(a, min_a = None, max_a = None):
	if min_a is None: min_a, max_a = np.min(a, axis = 0), np.max(a, axis = 0)
	return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a

In [18]:
train, min_a, max_a = normalize(X_train)

In [19]:
test, _, _ = normalize(X_test, min_a, max_a)

In [20]:
labels = np.repeat(y_test[:, np.newaxis], repeats=test.shape[1], axis=1)
labels = labels.astype(float)

In [21]:
labels

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [22]:
test_data = test
true_labels = labels

fig, axes = plt.subplots(test_data.shape[1], 1, figsize=(15, test_data.shape[1]*2), sharex=True)
time_steps = np.arange(test_data.shape[0])

for i in range(test_data.shape[1]):
    axes[i].plot(time_steps, test_data[:, i])
    axes[i].fill_between(time_steps, 0, 1, where=true_labels[:, i] == 1, color='red', alpha=0.2, label='Anomaly')
    axes[i].set_title(f'Channel {i}', fontsize=12)
    axes[i].set_xlabel('Time', fontsize=10)
    axes[i].set_ylabel('Value', fontsize=10)
    axes[i].grid(True)

fig.suptitle('Testing data with anomaly', fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Windows construction

In [23]:
def convert_to_windows(data, model):
	windows = []; w_size = model.n_window
	for i, g in enumerate(data): 
		if i >= w_size: w = data[i-w_size:i]
		else: w = torch.cat([data[0].repeat(w_size-i, 1), data[0:i]])
		windows.append(w)
	return torch.stack(windows)

### Dataloader

In [24]:
def data_loader(train, test, labels):
	# if args.less: loader[0] = cut_array(0.2, loader[0])
	train_loader = DataLoader(train, batch_size=train.shape[0])
	test_loader = DataLoader(test, batch_size=test.shape[0])
	labels_loader = labels
	return train_loader, test_loader, labels_loader

### Model

In [25]:
def create_model(modelname, dims):
	import src.models
	model_class = getattr(src.models, modelname)
	model = model_class(dims).double()

	optimizer = torch.optim.AdamW(model.parameters() , lr=model.lr, weight_decay=1e-5)
	scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.9)

	print(f"{color.GREEN}Creating new model: {model.name}{color.ENDC}")
	
	epoch = -1
	accuracy_list = []

	return model, optimizer, scheduler, epoch, accuracy_list

### Training function

In [26]:
def backprop(epoch, model, data, dataO, optimizer, scheduler, training = True):
    feats = dataO.shape[1]
    l = nn.MSELoss(reduction = 'none')
    
    data_x = torch.DoubleTensor(data); dataset = TensorDataset(data_x, data_x)
    bs = model.batch if training else len(data)
    dataloader = DataLoader(dataset, batch_size = bs)
    n = epoch + 1; w_size = model.n_window
    l1s, l2s = [], []
    if training:
        for d, _ in dataloader:
            local_bs = d.shape[0]
            window = d.permute(1, 0, 2)
            elem = window[-1, :, :].view(1, local_bs, feats)
            z = model(window, elem)
            l1 = l(z, elem) if not isinstance(z, tuple) else (1 / n) * l(z[0], elem) + (1 - 1/n) * l(z[1], elem)
            if isinstance(z, tuple): z = z[1]
            l1s.append(torch.mean(l1).item())
            loss = torch.mean(l1)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        scheduler.step()
        tqdm.write(f'Epoch {epoch},\tL1 = {np.mean(l1s)}')
        return np.mean(l1s), optimizer.param_groups[0]['lr']
    else:
        for d, _ in dataloader:
            window = d.permute(1, 0, 2)
            elem = window[-1, :, :].view(1, bs, feats)
            z = model(window, elem)
            if isinstance(z, tuple): z = z[1]
        loss = l(z, elem)[0]
        return loss.detach().numpy(), z.detach().numpy()[0]

### Model training

In [27]:
train_loader, test_loader, labels_loader = data_loader(train, test, labels)

In [28]:
model, optimizer, scheduler, epoch, loss_list = create_model('TranAD', labels_loader.shape[1])

Creating new model: TranAD


In [29]:
## Prepare data
trainD, testD = next(iter(train_loader)), next(iter(test_loader))
trainO, testO = trainD, testD 

trainD, testD = convert_to_windows(trainD, model), convert_to_windows(testD, model)

In [30]:
### Training phase
print(f'{color.HEADER}Training TranAD on external dataset.{color.ENDC}')
num_epochs = 5; e = epoch + 1; start = time()
for e in tqdm(list(range(epoch+1, epoch+num_epochs+1))):
    lossT, lr = backprop(e, model, trainD, trainO, optimizer, scheduler)
    loss_list.append((lossT, lr))
print(color.BOLD+'Training time: '+"{:10.4f}".format(time()-start)+' s'+color.ENDC)

  0%|          | 0/5 [00:00<?, ?it/s]

Training TranAD on external dataset.


 20%|██        | 1/5 [00:10<00:43, 10.88s/it]

Epoch 0,	L1 = 0.12148485558855007


 40%|████      | 2/5 [00:21<00:32, 10.77s/it]

...

Epoch 2,	L1 = 0.07505783503475488


 80%|████████  | 4/5 [00:43<00:10, 10.83s/it]

Epoch 3,	L1 = 0.05340863393003419


100%|██████████| 5/5 [00:53<00:00, 10.64s/it]

Epoch 4,	L1 = 0.0386850490250881
Training time:    53.2174 s


### Model evaluation

In [31]:
import gc
gc.collect()

72

In [32]:
### Testing phase
torch.zero_grad = True
model.eval()
print(f'{color.HEADER}Testing TranAD on external dataset{color.ENDC}')
loss, y_pred = backprop(0, model, testD, testO, optimizer, scheduler, training=False)

### Scores
result_df = pd.DataFrame()
lossT, _ = backprop(0, model, trainD, trainO, optimizer, scheduler, training=False)
for i in range(loss.shape[1]):
    lt, l, ls = lossT[:, i], loss[:, i], labels_loader[:, i]
    result, pred = pot_eval(lt, l, ls); preds.append(pred)
    result_df = result_df.append(result, ignore_index=True)

lossTfinal, lossFinal = np.mean(lossT, axis=1), np.mean(loss, axis=1)
labelsFinal = (np.sum(labels_loader, axis=1) >= 1) + 0
result, _ = pot_eval(lossTfinal, lossFinal, labelsFinal)
result.update(hit_att(loss, labels_loader))
result.update(ndcg(loss, labels_loader))
print(result_df)
pprint(result)

Testing TranAD on external dataset
       FN     FP   ROC/AUC       TN   TP        f1  precision    recall  \
0  1262.0   22.0  0.499657  32028.0  0.0  0.000000   0.000000  0.000000   
1  1262.0  123.0  0.498081  31927.0  0.0  0.000000   0.000000  0.000000   
2  1262.0   19.0  0.499704  32031.0  0.0  0.000000   0.000000  0.000000   
3  1261.0   12.0  0.500209  32038.0  1.0  0.001568   0.076923  0.000792   
4  1262.0    4.0  0.499938  32046.0  0.0  0.000000   0.000000  0.000000   
5  1255.0   42.0  0.502118  32008.0  7.0  0.010678   0.142857  0.005547   

   threshold  
0   0.315064  
1   0.337732  
2   0.285833  
3   0.097683  
4   0.240656  
5   0.751634  
{'FN': 1257,
 'FP': 35,
 'Hit@100%': 1.0,
 'Hit@150%': 1.0,
 'NDCG@100%': 1.0,
 'NDCG@150%': 1.0,
 'ROC/AUC': 0.5014349607264798,
 'TN': 32015,
 'TP': 5,
 'f1': 0.007679895916981812,
 'precision': 0.12499996875000781,
 'recall': 0.00396196510331248,
 'threshold': 0.1689908869532001}


In [33]:
final_preds = (np.sum(np.array(preds).T, axis=1) >= 1) + 0

In [34]:
y_test.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [35]:
def calc_point2point(predict, actual):
    """
    calculate f1 score by predict and actual.
    Args:
        predict (np.ndarray): the predict label
        actual (np.ndarray): np.ndarray
    """
    TP = np.sum(predict * actual)
    TN = np.sum((1 - predict) * (1 - actual))
    FP = np.sum(predict * (1 - actual))
    FN = np.sum((1 - predict) * actual)
    precision = TP / (TP + FP + 0.00001)
    recall = TP / (TP + FN + 0.00001)
    f1 = 2 * precision * recall / (precision + recall + 0.00001)
    try:
        roc_auc = roc_auc_score(actual, predict)
    except:
        roc_auc = 0
    return f1, precision, recall, TP, TN, FP, FN, roc_auc

In [36]:
eval_result = calc_point2point(final_preds, y_test.values)

In [37]:
print(f'"f1": {eval_result[0]}\n"precision": {eval_result[1]}\n"recall": {eval_result[2]}\n"TP": {eval_result[3]}\n"TN": {eval_result[4]}\n"FP": {eval_result[5]}\n"FN": {eval_result[6]}\n"ROC/AUC": {eval_result[7]}')

"f1": 0.01079369354425212
"precision": 0.03636363471074388
"recall": 0.006339144165299967
"TP": 8
"TN": 31838
"FP": 212
"FN": 1254
"ROC/AUC": 0.49986223981447375


In [38]:
test_data = test
true_labels = labels
predicted_labels = np.array(preds).T + 0

fig, axes = plt.subplots(test_data.shape[1], 1, figsize=(15, test_data.shape[1]*2), sharex=True)
time_steps = np.arange(test_data.shape[0])

for i in range(test_data.shape[1]):
    axes[i].plot(time_steps, test_data[:, i])
    axes[i].fill_between(time_steps, 0, 0.5, where=true_labels[:, i] == 1, color='red', alpha=0.2, label='True Anomaly')
    axes[i].fill_between(time_steps, 0.5, 1, where=predicted_labels[:, i] == 1, color='blue', alpha=0.3, label='Predicted Anomaly')
    axes[i].set_title(f'Channel {i}', fontsize=12)
    axes[i].set_xlabel('Time', fontsize=10)
    axes[i].set_ylabel('Value', fontsize=10)
    axes[i].grid(True)
    axes[i].legend()

fig.suptitle('Testing data with true anomaly and predicted anomaly', fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()